# Subset Creator
@author: Max Felius

In [1]:
#imports
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os, sys, time
import re, datetime
from scipy import spatial
from tqdm import tqdm
from decouple import config

#import personal functions
sys.path.extend(os.path.join(os.getcwd(),'sinkhole_functions'))
from sinkhole_functions.functions import *
from sinkhole_functions.influence_functions import *
from sinkhole_functions.spatio_temporal_functions import *

In [ ]:
#import paths
mrss = config('MRSS')
filename = 'full-pixel_mrss_s1_asc_t88_v4_080a1cbf7de1b6d42b3465772d9065fe7115d4bf.csv'

#import data
data = pd.read_csv(os.path.join(mrss,filename))

In [8]:
#convert to rijksdriehoek
data_gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.pnt_lon, data.pnt_lat))
data_gdf.crs = 'epsg:4326' #wgs84
data_gdf = data_gdf.to_crs('epsg:28992')

#new locations coordinates in Rijksdriehoek
x0 = 203050
y0 = 318500

#make the subset
rdx = data_gdf.geometry.x.values
rdy = data_gdf.geometry.y.values
rd_data = np.concatenate((rdx.reshape(len(rdx),1),rdy.reshape(len(rdy),1)),axis=1)
tree = spatial.cKDTree(rd_data)

#cropping the dataset
radius = 100
subset = tree.query_ball_point(([x0,y0]),r=radius)
data = data.iloc[subset]

#save dataset
data.to_csv('Subset_MRSS_s1_asc_t88_x0_{:.02f}_y0_{:.02f}_r{}m.csv'.format(x0,y0,radius))